In [5]:
import os

while 'resco_benchmark' not in os.listdir():
    os.chdir('..')

from resco_benchmark.multi_signal import MultiSignal
from resco_benchmark.config.agent_config import agent_configs
from resco_benchmark.config.map_config import map_configs
from resco_benchmark.config.mdp_config import mdp_configs

args = agent_configs['IDQN']
map_name = 'ingolstadt1'
map_config = map_configs[map_name]

env = MultiSignal(run_name = 'test', map_name = map_name, 
                  net = os.path.join('resco_benchmark/', map_config['net']), state_fn = args['state'], 
                  reward_fn = args['reward'], route=None, gui=False, 
                  end_time=3600, step_length=10, yellow_length=4, step_ratio=1, 
                  max_distance=200, lights=(), log_dir='/', libsumo=False, warmup=0, gymma=False)

ingolstadt1 resco_benchmark/environments/ingolstadt1/ingolstadt1.sumocfg drq_norm wait_norm
 Retrying in 1 seconds
lights 1 ('gneJ207',)
Step #57600.00 (0ms ?*RT. ?UPS, TraCI: 5ms, vehicles TOT 0 ACT 0 BUF 0)                   


OSError: [Errno 30] Read-only file system: '/test-ingolstadt1-0-drq_norm-wait_norm'

In [4]:
map_config['net']

'environments/ingolstadt1/ingolstadt1.sumocfg'

In [3]:
import os
while 'resco_benchmark' not in os.listdir():
    os.chdir('..')
!ls

Creative Commons — Attribution-NonCommercial-ShareAlike 4.0 International — CC BY-NC-SA 4.0.pdf
MANIFEST.in
README.md
delays.png
maps.png
resco_benchmark
setup.py


'/Users/pawelnarkiewicz/Documents/GitHub/RESCO_Test'

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# REMARK: no activation function in GraphConvLayer!!!
class GraphConvLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GraphConvLayer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x, adj_matrix):
        """
        x: [B, NUM_NODES, INPUT_DIM]
        adj_matrix: [NUM_NODES, NUM_NODES]

        output: [B, NUM_NODES, OUTPUT_DIM]
        """

        batch_size, num_nodes, _ = x.size()

        adj_matrix_hat = adj_matrix + torch.eye(n=num_nodes)
        deg_mat_inv_square_root = torch.diag(torch.sum(adj_matrix, dim=-1)** (-0.5))
        adj_matrix_modified = deg_mat_inv_square_root @ adj_matrix_hat @ deg_mat_inv_square_root
        
        output = self.linear(adj_matrix_modified @ x)
        
        return output

class GraphNeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GraphNeuralNetwork, self).__init__()
        assert num_layers >= 2, "Number of layers needs to be greater or equal to 2"
        self.num_middle_layers = num_layers-2
        self.first_layer = GraphConvLayer(input_dim, hidden_dim)
        self.middle_layers = nn.ModuleList([GraphConvLayer(hidden_dim, hidden_dim) for _ in range(self.num_middle_layers)])
        self.last_layer = GraphConvLayer(hidden_dim, output_dim)

    def forward(self, x, adj_matrix):
        """
        x: [B, NUM_NODES, INPUT_FEATURES]
        adj_matrix: [NUM_NODES, NUM_NODES]

        output: [B, NUM_NODES, OUTPUT_DIM]
        """
        x = F.relu(self.first_layer(x, adj_matrix))
        for i in range(self.num_middle_layers):
            x = F.relu(self.middle_layers[i](x, adj_matrix))
        output = self.last_layer(x, adj_matrix)
        return output

# Correctness test
input_dim = 16
hidden_dim = 32
output_dim = 2
batch_size = 10
num_nodes = 5
num_layers = 6

x = torch.randn(batch_size, num_nodes, input_dim)
adj_matrix = torch.randint(0, 2, (num_nodes, num_nodes)).float()

gnn = GraphNeuralNetwork(input_dim, hidden_dim, output_dim, num_layers)

output = gnn(x, adj_matrix)
print(output.shape)


torch.Size([10, 5, 2])


In [6]:
!pip install pfrl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pfrl: filename=pfrl-0.4.0-py3-none-any.whl size=155461 sha256=86e102835bcab5647941cc041b572ed9901d61fbc40d68742f81493b3905a8dd
  Stored in directory: /Users/pawelnarkiewicz/Library/Caches/pip/wheels/22/4a/0f/a87cd1ae925086eb3a1b8759f620fcf48e47939fb082946c3b
Successfully built pfrl
